### testing fill missing

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pathlib
import time
import pyspark.pandas as ps

from datetime import date, datetime, timedelta
from pyspark.pandas.indexes.datetimes import DatetimeIndex
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import when, col, rank, lit, monotonically_increasing_id, date_trunc, udf, min, max, explode
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, FloatType, DateType, StringType

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/08 22:40:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/08 22:40:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
glucose_data_schema=StructType([StructField('NumId', IntegerType(), True),
                                        StructField('PatientId', StringType(), True),
                                        StructField('Value', FloatType(), True),
                                        StructField('GlucoseDisplayTime', TimestampType(), True),
                                        StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                        StructField('GlucoseDisplayDate', DateType(), True)])

In [5]:
startTime = time.time()
data_location = "/cephfs/train_test_val/train_set"
allPaths = [str(x) for x in list(pathlib.Path(data_location).glob('*.parquet')) if 'part-00' in str(x)][0]
#allPaths.sort()
# print(allPaths)

pyspark_glucose_data = spark.read \
                       .schema(glucose_data_schema) \
                       .format('parquet') \
                       .load(allPaths)
pyspark_glucose_data = pyspark_glucose_data.withColumn("GlucoseDisplayTime",
                                                       date_trunc("minute",
                                                       col("GlucoseDisplayTime")))

pyspark_glucose_data=pyspark_glucose_data.orderBy("PatientId",
                                                  "GlucoseDisplayTime",
                                                  ascending=True)

print(time.time()-startTime)

3.7980234622955322


In [ ]:
test = pyspark_binary_labels(pyspark_glucose_data)

In [6]:
df = pyspark_glucose_data.toPandas()

/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [7]:
df.head(10)

,NumId,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayTimeRaw,GlucoseDisplayDate
0,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,239.0,2022-02-01 00:01:00,2022-02-01T00:01:25.000-06:00,2022-02-01
1,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,229.0,2022-02-01 00:06:00,2022-02-01T00:06:25.000-06:00,2022-02-01
2,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,219.0,2022-02-01 00:11:00,2022-02-01T00:11:26.000-06:00,2022-02-01
3,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,211.0,2022-02-01 00:16:00,2022-02-01T00:16:25.000-06:00,2022-02-01
4,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,202.0,2022-02-01 00:21:00,2022-02-01T00:21:25.000-06:00,2022-02-01
5,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,198.0,2022-02-01 00:26:00,2022-02-01T00:26:26.000-06:00,2022-02-01
6,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,191.0,2022-02-01 00:31:00,2022-02-01T00:31:26.000-06:00,2022-02-01
7,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,184.0,2022-02-01 00:36:00,2022-02-01T00:36:26.000-06:00,2022-02-01
8,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,178.0,2022-02-01 00:41:00,2022-02-01T00:41:25.000-06:00,2022-02-01
9,6782,48tvP/+WswO0u4OPaFoSqyW8rPDU4Nu/w7J4ctOnCo4=,173.0,2022-02-01 00:46:00,2022-02-01T00:46:26.000-06:00,2022-02-01


In [9]:
lower_10, upper_90 = df.Value.quantile([.1, .9], interpolation='nearest')

print(lower_10)
print(upper_90)

96.0
264.0


In [10]:
default_lower = 70
default_upper = 180

lower = lower_10 if lower_10 < default_lower else default_lower
upper = upper_90 if upper_90 > default_upper else default_upper

print(lower)
print(upper)

70
264.0


In [12]:
df['y_Binary'] = [1 if ((x > upper) or (x < lower)) else 0 for x in df['Value']]

In [17]:
# def pyspark_binary_labels(self, df):
#         #get 10th and 90th percentiles of patient
#         lower_10, upper_90  = df.approxQuantile('Value', [.1, .9], 0)
        
#         #if 10th percentile of patient is lower than default, use percentile
#         lower = lower_10 if lower_10 < self.lower else self.lower
#         upper = upper_90 if upper_90 > self.upper else self.upper
        
#         df=df.withColumn('y_Binary', F.when(F.col('Value') > upper, 1)\
#             .when(F.col('Value') < lower, 1)\
#                 .otherwise(0))
#         df=df.withColumn('is_above', F.when(F.col('Value') > upper, 1).otherwise(0))
#         df=df.withColumn('is_below', F.when(F.col('Value') < lower, 1).otherwise(0))

#         return df
    
def pyspark_binary_labels(self, df):
    @pandas_udf(self.outSchema, functionType=PandasUDFType.GROUPED_MAP)
    def pandas_binary_labels(self, df):
        lower_10, upper_90 = df.Value.quantile([.1, .9], interpolation='nearest')
        lower = lower_10 if lower_10 < self.lower else self.lower
        upper = upper_90 if upper_90 > self.upper else self.upper
        df['y_Binary'] = [1 if ((x > upper) or (x < lower)) else 0 for x in df['Value']]
        df['is_above'] = [1 if (x > upper) else 0 for x in df['Value']]
        df['is_below'] = [1 if (x < lower) else 0 for x in df['Value']]
            
        return df  
    
    return df.groupBy('PatientId').apply(pyspark_binary_labels)

In [ ]:
test_df = pyspark_glucose_data

In [ ]:
test_df.show(4)

In [ ]:
min_max = test_df.groupBy('NumId')\
    .agg(min("GlucoseDisplayTime").alias('MinDate'),\
        max("GlucoseDisplayTime").alias('MaxDate'))

In [ ]:
min_max = min_max.withColumn("Index", monotonically_increasing_id())\
.select('Index','NumId','MinDate','MaxDate')

In [ ]:
min_max

In [ ]:
merged_schema = StructType([StructField('NumId', IntegerType(), True),
                            StructField('GlucoseDisplayTime', TimestampType(), True)])

In [ ]:
merge_df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema=merged_schema)

for idx in range(min_max.count()):
    row = min_max.filter(min_max.Index == idx).first()
    date_range = pd.DataFrame(pd.Series(ps.date_range(row.MinDate, row.MaxDate, freq='5min')))
    start = time.time()
    merge_df = spark.createDataFrame(date_range.to_numpy(), \
                                     schema=StructType([StructField('GlucoseDisplayTime', TimestampType(), True)]))
    print(time.time() - start)
    merge_df = merge_df.withColumn('NumId', lit(row.NumId))
    #merge_df = merge_df.withColumn('GlucoseDisplayTime', date_range)                                                           
    
    print(merge_df)
    break;

In [ ]:
def date_range_def(minDate, maxDate):    
    return ps.date_range(minDate, maxDate, freq='5min')

date_range_udf = udf(lambda x,y: date_range_def(x,y))

In [ ]:
min_max = min_max.withColumn('Range', date_range_udf(col('MinDate'), col('MaxDate')))

In [ ]:
min_max.show(5)

In [ ]:
dates = spark.sql("SELECT sequence(CAST('2022-04-08 07:49:00' AS DATE), CAST('2022-10-01 07:47:00' AS DATE), interval 5 minutes) as date").withColumn("date", explode(col("date")))

In [ ]:
dates.show()

In [ ]:
@pandas_udf(PandasUDFType.GROUPED_MAP)
def replace_missing(subset):
    """ INPUT
        subset:     spark DataFrame with 1 patient
        OUTPUT
        filled_df:  spark DataFrame with 1 patient (-1 columns) and all missing rows filled in; not sorted
    """

    '''get first and last date (takes about 10 seconds per ten days of one patient)'''
    minimum = subset.agg({'GlucoseDisplayTime': 'min'}).collect()[0][0]
    maximum = subset.agg({'GlucoseDisplayTime': 'max'}).collect()[0][0]

    '''make a range that fills all those in'''
    def date_range_list(start_date, end_date):
        if start_date > end_date:
            raise ValueError("start_date must come before end_date")

        datetime_list = []
        curr_date = start_date
        while curr_date <= end_date:
            datetime_list.append([curr_date])
            curr_date += timedelta(minutes=5)
        return datetime_list

    datetime_list = date_range_list(minimum, maximum)

    '''make a dataframe of those dates'''
    deptSchema = StructType([       
        StructField('GlucoseDisplayTime', TimestampType(), True)
    ])
    dt_df = self.spark.createDataFrame(data=datetime_list, schema=deptSchema)

    '''merge og dataframe back into the new one'''
    merged = subset.unionByName(dt_df, allowMissingColumns=True)

    '''get rid of the timestamps we already have (using the exact same method as from "drop duplicate datetimes for each patient" above)'''
    window = Window.partitionBy('GlucoseDisplayTime').orderBy('tiebreak')
    merged = (merged
     .withColumn('tiebreak', monotonically_increasing_id())
     .withColumn('rank', rank().over(window))
     .filter(col('rank') == 1).drop('rank','tiebreak')
    )

    '''filling out the columns as needed:
        -PatientId should be all the same string
        -GlucoseDisplayTimeRaw should be used for checking the dates here, but implementation will have to come later'''
    merged = merged.fillna(patient_str, subset='PatientId')
    merged = merged.drop('GlucoseDisplayTimeRaw') #someday i'll have time to use this as the double-checker
    merged = merged.withColumn('GlucoseDisplayDate',
                               to_date(col('GlucoseDisplayTime')))

    """ ============== FILL IN MISSING VALUES ============== """
    # filler = subset.agg({'Value': 'median'}).collect()[0][0]
    filler = subset.agg({'Value': 'avg'}).collect()[0][0]

    filled_df = merged.fillna(filler, subset='Value')

    return filled_df
    

In [ ]:
test = test_df.groupBy('NumId').apply(lambda x: replace_missing(x))